In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from sample.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from sample.notebooks.analysis.data_info import *


In [2]:
exp = get_exp((39,269,329),directory_project)


begin = 2020-11-23 15:01:00 
  end = 2020-12-01 10:09:00


FileNotFoundError: [Errno 2] No such file or directory: '/projects/0/einf914/data//Analysis_Plate39_2020-11-23 15:01:00_2020-12-01 10:09:00/experiment_39.pick'

In [ ]:
def get_hyph_infos(exp):
    select_hyph = {}
    for hyph in exp.hyphaes:
        select_hyph[hyph] = []
        for i,t in enumerate(hyph.end.ts[:-1]):
            tp1=hyph.ts[i+1]
            pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
            speed = np.sum([get_length_um(seg) for seg in pixels])/get_time(exp,t,tp1)
            select_hyph[hyph].append((t,hyph.ts[i+1],speed,pixels))
    return(select_hyph)

In [3]:
select_hyph = get_hyph_infos(exp)

failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure 

In [4]:
rh2 = [hyph for hyph in exp.hyphaes if np.any(np.array([c[2] for c in select_hyph[hyph]])>=300)]

In [145]:
hyph = [rh for rh in rh2 if rh.end.label == 1][0]

In [147]:
# hyph = choice(rh2)
speeds = [c[2] for c in select_hyph[hyph]]
ts = [c[0] for c in select_hyph[hyph]]
tp1s = [c[1] for c in select_hyph[hyph]]


In [148]:
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.plot(ts,speeds)
ax.set_xlabel('time (h)')
ax.set_ylabel('speed ($\mu m .h^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'speed ($\\mu m .h^{-1}$)')

In [149]:
plot_raw_plus(exp,hyph.ts[-1],[hyph.end.label]+[hyph.root.label])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
counts = []
for t in range(exp.ts):
    count = 0
    for hyph in rh2:
        if int(hyph.end.ts()[-1])==int(t):
            count+=1
    counts.append(count)

In [105]:
counts

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 3,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 32]

In [137]:
plot_raw_plus(exp,hyph.ts[-1]+1,[hyph.end.label]+[hyph.root.label]+[5107,5416])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
nx_graph_t = exp.nx_graph[35]
nx_graph_tm1 = exp.nx_graph[34]
Sedge = sparse.csr_matrix((30000, 60000))
for edge in nx_graph_t.edges:
    pixel_list = nx_graph_t.get_edge_data(*edge)["pixel_list"]
    pixela = pixel_list[0]
    pixelb = pixel_list[-1]
    Sedge[pixela[0], pixela[1]] = edge[0]
    Sedge[pixelb[0], pixelb[1]] = edge[1]
tip = 2326
pos_tm1 = exp.positions[34]
pos_t = exp.positions[35]

mini1 = np.inf
posanchor = pos_tm1[tip]
window = 1000
potential_surrounding_t = Sedge[
    max(0, posanchor[0] - 2 * window) : posanchor[0] + 2 * window,
    max(0, posanchor[1] - 2 * window) : posanchor[1] + 2 * window,
]
#         potential_surrounding_t=Sedge
#         for edge in nx_graph_t.edges:
#             pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
#             if np.linalg.norm(np.array(pixel_list[0])-np.array(pos_tm1[tip]))<=5000:
#                 distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
#                 if distance<mini1:
#                     mini1=distance
#                     right_edge1 = edge
#         print('t1 re',right_edge)
mini = np.inf
for node_root in potential_surrounding_t.data:
    for edge in nx_graph_t.edges(int(node_root)):
        pixel_list = nx_graph_t.get_edge_data(*edge)["pixel_list"]
        if (
            np.linalg.norm(np.array(pixel_list[0]) - np.array(pos_tm1[tip]))
            <= 5000
        ):
            distance = np.min(
                np.linalg.norm(
                    np.array(pixel_list) - np.array(pos_tm1[tip]), axis=1
                )
            )
            if distance < mini:
                mini = distance
                right_edge = edge

In [138]:
right_edge,mini

((5107, 5416), 9.433981132056603)

In [143]:
origin = np.array(
            orient(
                nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])[
                    "pixel_list"
                ],
                pos_tm1[tip],
            )
        )
origin_vector = origin[0] - origin[-1]
branch = np.array(
    orient(
        nx_graph_t.get_edge_data(*right_edge)["pixel_list"],
        pos_t[right_edge[0]],
    )
)
candidate_vector = branch[-1] - branch[0]
dot_product = np.dot(origin_vector, candidate_vector)
if dot_product >= 0:
    root = right_edge[0]
    next_node = right_edge[1]
else:
    root = right_edge[1]
    next_node = right_edge[0]
last_node = root
current_node = next_node
last_branch = np.array(
    orient(
        nx_graph_t.get_edge_data(root, next_node)["pixel_list"],
        pos_t[current_node],
    )
)
i = 0
loop = []
while (
    nx_graph_t.degree(current_node) != 1
    and not current_node in nx_graph_tm1.nodes
):  # Careful : if there is a cycle with low angle this might loop indefinitely but unprobable
    i += 1
    if i >= 100:
        print(
            "identified infinite loop",
            i,
            tip,
            current_node,
            pos_t[current_node],
        )
        break
    mini = np.inf
    origin_vector = (
        last_branch[0] - last_branch[min(length_id, len(last_branch) - 1)]
    )
    unit_vector_origin = origin_vector / np.linalg.norm(origin_vector)
    candidate_vectors = []
    for neighbours_t in nx_graph_t.neighbors(current_node):
        if neighbours_t != last_node:
            branch_candidate = np.array(
                orient(
                    nx_graph_t.get_edge_data(current_node, neighbours_t)[
                        "pixel_list"
                    ],
                    pos_t[current_node],
                )
            )
            candidate_vector = (
                branch_candidate[min(length_id, len(branch_candidate) - 1)]
                - branch_candidate[0]
            )
            unit_vector_candidate = candidate_vector / np.linalg.norm(
                candidate_vector
            )
            candidate_vectors.append(unit_vector_candidate)
            dot_product = np.dot(unit_vector_origin, unit_vector_candidate)
            angle = np.arccos(dot_product)
            if angle < mini:
                mini = angle
                next_node = neighbours_t
    if len(candidate_vectors) < 2:
        print(
            "candidate_vectors < 2",
            nx_graph_t.degree(current_node),
            pos_t[current_node],
            [node for node in nx_graph_t.nodes if nx_graph_t.degree(node) == 2],
        )
    competitor = np.arccos(np.dot(candidate_vectors[0], -candidate_vectors[1]))
    if mini < competitor:
        current_node, last_node = next_node, current_node

In [144]:
current_node

5107